<a href="https://colab.research.google.com/github/noamarko/Neural-Networks/blob/main/HW2_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import pandas as pd

In [2]:
def BTU_layer(x, w, b, Temp = 0.1):    
  z = tf.matmul(x, w) + b
  return tf.nn.sigmoid(z / Temp) 

In [3]:
class XOR_Net_Model:
  def __init__(self, input_dim, num_hidden, num_hbridge, num_outputs, ShortCut):
    self.w1 = tf.Variable(tf.random.uniform([dim, num_hidden], -1, 1, seed=0), name="weights1", trainable=True)
    self.w2 = tf.Variable(tf.random.uniform([num_hbridge, num_outputs], -1, 1, seed=0), name="weights2", trainable=True)
    self.b1 = tf.Variable(tf.zeros([num_hidden]), name="bias1", trainable=True)
    self.b2 = tf.Variable(tf.zeros([num_outputs]), name="bias2", trainable=True)
    self.ShortCut = ShortCut
    
  # feed forward
  def __call__(self, x_train):
    self.hlayer1 = BTU_layer(x_train, self.w1, self.b1)
    if self.ShortCut:
        self.hlayer1 = tf.concat([self.hlayer1, x_train], 1)
    self.out = BTU_layer(self.hlayer1, self.w2, self.b2)
    return self.out

In [4]:
# compute loss on feed forward step
def Loss(out, t_train):
  loss = -tf.reduce_sum(t_train * tf.math.log(out) + (1.0 - t_train) * tf.math.log(1.0 - out))  # Cross Entropy loss function
  return tf.divide(loss, len(out), name="loss")
# compute grads for back propagation
def grad(model, x_train, t_train):
  with tf.GradientTape() as t:
    loss = Loss(model(x_train), t_train)
  return t.gradient(loss, [model.w1, model.b1, model.w2, model.b2]), loss

In [5]:
# 1 train step operation
def epoch(model, x_train, t_train, optimizer):
  grads, loss = grad(model, x_train, t_train)
  optimizer.apply_gradients(zip(grads, [model.w1, model.b1, model.w2, model.b2]))
  return loss

In [6]:
# run training:
def train(model, x_train, t_train, optimizer, x_test, t_test, num_epochs):
  ten_last_losses = [np.Infinity]*10 #array that will check if the condition = true
  i = 0
  for i in range(num_epochs+1):
    train_loss = epoch(model, x_train, t_train, optimizer).numpy()
    valid_loss = Loss(model(x_test), t_test).numpy()
    #if i % 1000 == 0:
    #  print("\n Valid Loss: ", valid_loss)  
    ten_last_losses[i%10] = valid_loss
    if tf.math.is_nan(valid_loss):
      return i, valid_loss, -1
    if i >= 9 and (valid_loss < 0.2) and (ten_last_losses[(i+1)%10] -  ten_last_losses[i%10]) < 0.0001:
      return i, valid_loss, train_loss
  return i,valid_loss,-1

In [7]:
def test(model, x_test, t_test):
  loss = Loss(model(x_test), t_test)
  return loss

In [8]:
def start_exp(dim, num_hidden, num_outputs, l_rate, ShortCut, num_epochs):
  
  epochs = []
  v_losses = []
  t_losses = []
  i = 0
  fail = 0
  while i < 10:
    if ShortCut:
        num_hbridge = num_hidden + dim
    else:
        num_hbridge = num_hidden
    # define a model:
    model = XOR_Net_Model(dim, num_hidden, num_hbridge, num_outputs, ShortCut)
    # set an optimizer:
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(l_rate)
    #defining the train group and the validation group
    x_train = tf.convert_to_tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=tf.float32)
    t_train = tf.convert_to_tensor([[0], [1], [1], [0]], dtype=tf.float32)
    x_test = tf.convert_to_tensor([[0, 0], [0, 1], [1, 0], [1, 1], [1, 0.1], [1, 0.9], [0.9,0.9], [0.1, 0.9]], dtype=tf.float32)
    t_test = tf.convert_to_tensor([[0], [1], [1], [0], [1], [0], [0], [1]], dtype=tf.float32)
    num, CE_test, CE_train  = train(model, x_train, t_train, optimizer, x_test, t_test, num_epochs)
    if num != num_epochs and i<10 and np.logical_not(tf.math.is_nan(CE_test)):
      epochs.append(num)
      v_losses.append(CE_test)
      t_losses.append(CE_train)
      i += 1
      if num_hidden == 1:
        print(model.hlayer1)
    else:
      fail += 1
    
  mean_v = np.mean(v_losses)
  mean_t = np.mean(t_losses)
  mean_e = np.mean(epochs)
  std_v = np.std(v_losses)
  std_t = np.std(t_losses)
  std_e = np.std(epochs)

  print("\nmeanepocs: {},          std/epocs: {},   Failures: {}".format(mean_e, std_e, fail))
  print("\nmeanvalidloss: {},      stdvalidlossPercent: {}".format(mean_v, std_v))
  print("\nnmeanTrainLoss: {},     stdTrainLossPercent: {}".format(mean_t, std_t))
  return mean_e, std_e 

## Expirement 1:
Hidden Neurons: 2

Learning Rate: 0.1

Short Cut: True

In [9]:
epoch_mean = []
epoch_std = []
learning_r = []
hidden_n = []
bridge = []
dim = 2
num_hidden = 2
num_outputs = 1
l_rate = 0.1
ShortCut = True
num_epochs = 40000
print("Expirement 1:  Hidden: {},  LR: {},  Bridge: {}".format(num_hidden, l_rate, ShortCut))
ep_m, ep_std = start_exp(dim, num_hidden, num_outputs, l_rate, ShortCut, num_epochs)
epoch_mean.append(ep_m)
epoch_std.append(ep_std)



Expirement 1:  Hidden: 2,  LR: 0.1,  Bridge: True

meanepocs: 499.7,          std/epocs: 108.75941338569274,   Failures: 1

meanvalidloss: 0.00695795938372612,      stdvalidlossPercent: 0.0017819652566686273

nmeanTrainLoss: 0.003166498616337776,     stdTrainLossPercent: 0.00040444982005283237


## Expirement 2:
Hidden Neurons: 2

Learning Rate: 0.1

Short Cut:  False

In [ ]:
dim = 2
num_hidden = 2
num_outputs = 1
l_rate = 0.1
ShortCut = False
num_epochs = 40000
print("Expirement 2:  Hidden: {},  LR: {},  Bridge: {}".format(num_hidden, l_rate, ShortCut))
ep_m, ep_std = start_exp(dim, num_hidden, num_outputs, l_rate, ShortCut, num_epochs)
epoch_mean.append(ep_m)
epoch_std.append(ep_std)



Expirement 2:  Hidden: 2,  LR: 0.1,  Bridge: False


## Expirement 3:
Hidden Neurons: 2

Learning Rate: 0.01

Short Cut: True

In [ ]:
dim = 2
num_hidden = 2
num_outputs = 1
l_rate = 0.01
ShortCut = True
num_epochs = 40000
print("Expirement 3:  Hidden: {},  LR: {},  Bridge: {}".format(num_hidden, l_rate, ShortCut))
ep_m, ep_std = start_exp(dim, num_hidden, num_outputs, l_rate, ShortCut, num_epochs)
epoch_mean.append(ep_m)
epoch_std.append(ep_std)



## Expirement 4:
Hidden Neurons: 2

Learning Rate: 0.01

Short Cut: False

In [ ]:
dim = 2
num_hidden = 2
num_outputs = 1
l_rate = 0.01
ShortCut = False
num_epochs = 40000
print("Expirement 4:  Hidden: {},  LR: {},  Bridge: {}".format(num_hidden, l_rate, ShortCut))
ep_m, ep_std = start_exp(dim, num_hidden, num_outputs, l_rate, ShortCut, num_epochs)
epoch_mean.append(ep_m)
epoch_std.append(ep_std)



## Expirement 5:
Hidden Neurons: 4

Learning Rate: 0.1

Short Cut: True

In [ ]:
dim = 2
num_hidden = 4
num_outputs = 1
l_rate = 0.1
ShortCut = True
num_epochs = 40000
print("Expirement 5:  Hidden: {},  LR: {},  Bridge: {}".format(num_hidden, l_rate, ShortCut))
ep_m, ep_std = start_exp(dim, num_hidden, num_outputs, l_rate, ShortCut, num_epochs)
epoch_mean.append(ep_m)
epoch_std.append(ep_std)



## Expirement 6:
Hidden Neurons: 4

Learning Rate: 0.1

Short Cut: False

In [ ]:
dim = 2
num_hidden = 4
num_outputs = 1
l_rate = 0.1
ShortCut = False
num_epochs = 40000
print("Expirement 6:  Hidden: {},  LR: {},  Bridge: {}".format(num_hidden, l_rate, ShortCut))
ep_m, ep_std = start_exp(dim, num_hidden, num_outputs, l_rate, ShortCut, num_epochs)
epoch_mean.append(ep_m)
epoch_std.append(ep_std)



## Expirement 7:
Hidden Neurons: 4

Learning Rate: 0.01

Short Cut: True

In [ ]:
dim = 2
num_hidden = 4
num_outputs = 1
l_rate = 0.01
ShortCut = True
num_epochs = 40000
print("Expirement 7:  Hidden: {},  LR: {},  Bridge: {}".format(num_hidden, l_rate, ShortCut))
ep_m, ep_std = start_exp(dim, num_hidden, num_outputs, l_rate, ShortCut, num_epochs)
epoch_mean.append(ep_m)
epoch_std.append(ep_std)


## Expirement 8:
Hidden Neurons: 4

Learning Rate: 0.01

Short Cut: False

In [ ]:
dim = 2
num_hidden = 4
num_outputs = 1
l_rate = 0.01
ShortCut = False
num_epochs = 40000
print("Expirement 8:  Hidden: {},  LR: {},  Bridge: {}".format(num_hidden, l_rate, ShortCut))
ep_m, ep_std = start_exp(dim, num_hidden, num_outputs, l_rate, ShortCut, num_epochs)
epoch_mean.append(ep_m)
epoch_std.append(ep_std)



## Expirement 9:
Hidden Neurons: 1

Learning Rate: 0.01

Short Cut: True

In [ ]:
dim = 2
num_hidden = 1
num_outputs = 1
l_rate = 0.01
ShortCut = True
num_epochs = 40000
print("Expirement 9:  Hidden: {},  LR: {},  Bridge: {}".format(num_hidden, l_rate, ShortCut))
ep_m, ep_std = start_exp(dim, num_hidden, num_outputs, l_rate, ShortCut, num_epochs)
epoch_mean.append(ep_m)
epoch_std.append(ep_std)


## Creating Graphs

## 1. Epoch mean by number of hidden Neurons

In [ ]:
epoch_oneh = epoch_mean[8]
epoch_twoh = np.mean([epoch_mean[0], epoch_mean[1], epoch_mean[2], epoch_mean[3]])
epoch_fourh = np.mean([epoch_mean[4], epoch_mean[5], epoch_mean[6], epoch_mean[7]])
plt.axis([0, 4, 300, 4500])
plt.yscale('linear')
plt.ylabel('Epoch Means')
plt.xlabel('Hidden Neurons')
plt.plot([1, 2, 4] , [epoch_oneh, epoch_twoh, epoch_fourh])

## 2. Epoch mean by existence of bridge

In [ ]:
epochs_bridge = np.mean([epoch_mean[0], epoch_mean[2],epoch_mean[4], epoch_mean[6], epoch_mean[8]])
epochs_no_bridge = np.mean([epoch_mean[1], epoch_mean[3], epoch_mean[5], epoch_mean[7]])
df = pd.DataFrame({'Bridge':['Bridge', 'Without Bridge'], 'Epochs Mean':[epochs_bridge, epochs_no_bridge]})
ax = sns.barplot(x="Bridge", y="Epochs Mean", data=df)
ax.set_ylim([0, 2000])

## 3. Epoch std by learning rate

In [ ]:
epochs_std_01 = np.mean([epoch_std[0],
                         epoch_std[1],
                         epoch_std[4],
                         epoch_std[5]])
epochs_std_001 = np.mean([epoch_std[2],
                          epoch_std[3],
                          epoch_std[6],
                          epoch_std[7],
                          epoch_std[8]])
df = pd.DataFrame({'Learning Rate': [0.1, 0.01], 'STD of Epochs':[epochs_std_01, 
                                                                  epochs_std_001]})
ax = sns.barplot(x = "Learning Rate", y = "STD of Epochs", data = df)